In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [132]:
df = pd.read_csv('bank-full.csv')

In [133]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [134]:
df = df.drop(['day', 'month'], axis = 1)

In [135]:
df = df[~((df.poutcome == 'unknown') & (df.pdays != -1))]

In [136]:
df['poutcome'] = df['poutcome'].map({'unknown': 'none'}).fillna(df['poutcome'])

In [137]:
filter = df.apply(lambda x: False if (sum(x.isin(['unknown'])) > 0) else True, axis = 1)

In [138]:
df = df[filter].reset_index(drop = True)

In [140]:
df.poutcome.value_counts()

none       23060
failure     4679
other       1750
success     1413
Name: poutcome, dtype: int64

In [141]:
df['y'] = df['y'].map({'no': 0, 'yes': 1})

In [142]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

In [143]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

19776 train examples
4945 validation examples
6181 test examples


In [144]:
def df_to_dataset(dataframe = df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('y')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [87]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [146]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
  return lambda feature: encoder(index(feature))

In [147]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [148]:
all_inputs = []
encoded_features = []

# Numeric features.
for header in ['balance', 'duration', 'campaign', 'previous']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

In [150]:
# 66 tokens for pdays
len(set(df.pdays.value_counts()))

66

In [151]:
# Categorical features encoded as integers.
for header in ['age', 'pdays']:
    age_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
    encoding_layer = get_category_encoding_layer(header, train_ds, dtype='int64',
                                                 max_tokens=66)
    encoded_age_col = encoding_layer(age_col)
    all_inputs.append(age_col)
    encoded_features.append(encoded_age_col)

In [152]:
# Categorical features encoded as string.
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string',
                                               max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

In [153]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [128]:
# rankdir='LR' is used to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [154]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
78/78 [==============================] - 1s 6ms/step - loss: 0.4356 - accuracy: 0.8498 - val_loss: 0.3407 - val_accuracy: 0.8542
Epoch 2/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3462 - accuracy: 0.8575 - val_loss: 0.3089 - val_accuracy: 0.8580
Epoch 3/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3254 - accuracy: 0.8614 - val_loss: 0.2980 - val_accuracy: 0.8655
Epoch 4/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3123 - accuracy: 0.8674 - val_loss: 0.2919 - val_accuracy: 0.8688
Epoch 5/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3113 - accuracy: 0.8691 - val_loss: 0.2890 - val_accuracy: 0.8702
Epoch 6/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3031 - accuracy: 0.8687 - val_loss: 0.2863 - val_accuracy: 0.8710
Epoch 7/10
78/78 [==============================] - 0s 3ms/step - loss: 0.3025 - accuracy: 0.8699 - val_loss: 0.2850 - val_accuracy: 0.8702
Epoch 8/10
78/78 [==

In [155]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

25/25 [==============================] - 0s 2ms/step - loss: 0.2718 - accuracy: 0.8770
Accuracy 0.8770425319671631


In [157]:
model.save('SVM_bankmarketing_model')
# reloaded_model = tf.keras.models.load_model('SVM_bankmarketing_model')

INFO:tensorflow:Assets written to: SVM_bankmarketing_model/assets
